# Cookbook: Database Manipulation

In this cookbook, we will explore a simple data processing example: reading a file, adding its data to a database, and querying it. It shows how we can efficiently manipulate data with agents.

# Install Requirements

We will use [Qwen-Agent](https://github.com/QwenLM/Qwen-Agent) in this book. For demonstration, we will need at least the `mcp` functionality of Qwen-Agent.

In [ ]:
!pip3 install -U "qwen-agent[gui,rag,code_interpreter,mcp]"
# `pip install -U qwen-agent` will install the minimal requirements.
# The optional requirements, specified in double brackets, are:
#   [gui] for Gradio-based GUI support;
#   [rag] for RAG support;
#   [code_interpreter] for Code Interpreter support;
#   [mcp] for MCP support.

In [2]:
!pip3 install -U uv
# We will use mcp servers that run with the `uvx` command, so `uv` is required.

  Using cached uv-0.7.10-py3-none-macosx_11_0_arm64.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 5.9 MB/s eta 0:00:00a 0:00:01


In [3]:
!npm --version
# We also use mcp servers that run with the `npx` command, so node.js should be installed.

10.7.0


# Create a Spreadsheet

We create a sample spreadsheet and save it as `scores.csv` for later use.

In [4]:
import csv

csv_file_name = 'scores.csv'
table = [
    {"id": 100, "score": 85},
    {"id": 101, "score": 90},
    {"id": 102, "score": 88},
    {"id": 103, "score": 95},
    {"id": 104, "score": 80},
    {"id": 105, "score": 92},
    {"id": 106, "score": 87},
    {"id": 107, "score": 100},
    {"id": 108, "score": 83},
    {"id": 109, "score": 98}
]

with open(csv_file_name, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['id', 'score'])
    writer.writeheader()
    for record in table:
        writer.writerow(record)

# Step 1: Create an Agent

Create an agent that is capable of 
- listing, reading and writing files (through the MCP server `@modelcontextprotocol/server-filesystem`).
- performing SQLite database manipulations (through the MCP server `mcp-server-sqlite`).

In [5]:
from qwen_agent.agents import Assistant
from qwen_agent.utils.output_beautify import typewriter_print
# `typewriter_print` prints streaming messages in a non-overlapping manner for a clear view.

In [2]:
import dotenv
# Load environment variables from a .env file if it exists
dotenv.load_dotenv()
DASHSCOPE_API_KEY = dotenv.get_key(dotenv.find_dotenv(), 'DASHSCOPE_API_KEY')

In [11]:
llm_cfg = {
    'model': 'qwen3-32b',
    'model_server': 'dashscope',
    'api_key': DASHSCOPE_API_KEY # **fill your dashscope api key here**

    # Use a model service compatible with the OpenAI API, such as vLLM or Ollama:
    # 'model': 'Qwen3-8B',
    # 'model_server': 'http://localhost:8000/v1',  # base_url, also known as api_base
    # 'api_key': 'EMPTY'
}

tools = [
    {
        "mcpServers": {
            # enumeration of mcp server configs
            "filesystem": {
                "command": "npx",
                "args": [
                    "-y",
                    "@modelcontextprotocol/server-filesystem",
                    '.',
                ]
            },
            "sqlite" : {
                "command": "uvx",
                "args": [
                    "mcp-server-sqlite",
                    "--db-path",
                    "scores.db"
                ]
            }
        }
    }
]

agent = Assistant(
    llm=llm_cfg, 
    function_list=tools
)

2025-06-04 10:33:36,813 - qwen_dashscope.py - 147 - INFO - Setting the dashscope api_key.
2025-06-04 10:33:36,946 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['filesystem', 'sqlite']
2025-06-04 10:33:36,954 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: filesystem
2025-06-04 10:33:37,691 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: sqlite
2025-06-04 10:33:46,599 - mcp_manager.py - 237 - INFO - Failed in listing MCP resource templates: Method not found


# Step 2: Read the Spreadsheet

The agent will read the csv table by calling the file reading tool.

In [12]:
messages = [
    {
        'role': 'user',
        'content': 'Show me the contents in scores.csv.'
    }
]

In [13]:
response_plain_text = ''
for ret_messages in agent.run(messages):
    # `ret_messages` will contain all subsequent messages, consisting of interleaved assistant messages and tool responses
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user wants to see the contents of scores.csv. Let me check which tool to use. The available functions include reading a file, so I should use filesystem-read_file. The parameters require the path to the file. Assuming scores.csv is in an allowed directory, I'll call that function with the path "scores.csv". I need to make sure the path is correct and accessible. If there's an issue, the function will return an error, but I'll proceed with the call as instructed.
[TOOL_CALL] filesystem-read_file
{"path": "scores.csv"}
[TOOL_RESPONSE] filesystem-read_file
id,score
100,85
101,90
102,88
103,95
104,80
105,92
106,87
107,100
108,83
109,98

[THINK]
Okay, let me look at the user's request again. They wanted to see the contents of scores.csv, and I used the filesystem-read_file tool to get it. The response shows the CSV data with id and score columns. Now, I need to present this information clearly.

First, I'll check if the data makes sense. The IDs are sequential from 100 to 

In [14]:
messages += ret_messages # extending the context with new `ret_messages`.

# Step 3: Create a Database Table

Next, we create a database table and insert all the data that the agent just read.

In [15]:
messages.append({'role': 'user', 'content': "Create a table in the database, and fill it with the contents in the previous table."})

In [16]:
response_plain_text = ''

for ret_messages in agent.run(messages):
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, let me figure out how to approach this. The user wants to create a table in the database and populate it with the data from the scores.csv file we just looked at.

First, I need to create the table. The CSV has two columns: 'id' and 'score'. Both are integers. So the SQL statement should define a table with those columns. Maybe name the table 'scores'.

Next, inserting the data. The CSV data is already parsed into rows. Each row after the header has an id and a score. I'll need to generate an INSERT statement for each of those rows.

Wait, the user mentioned using the previous table's contents. The assistant showed the CSV data as a table, so the user is referring to that. So the data to insert is the 10 records listed.

So first, call sqlite-create_table with the appropriate CREATE TABLE statement. Then, for each row, execute an INSERT. But doing multiple INSERTs might be tedious. Alternatively, use a single INSERT with multiple VALUES clauses. However, the sqlite-write_

In [17]:
messages += ret_messages

# Step 4: Query the Highest Score

We query the highest score in the database. You can also perform other queries.

In [18]:
messages += [{'role': 'user', 'content': 'Query the highest score from the database.'}]

In [19]:
response_plain_text = ''

for ret_messages in agent.run(messages):
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user wants to query the highest score from the database. Let me think about how to approach this.

First, I need to recall the structure of the database. The user created a table called 'scores' with columns 'id' and 'score'. The data was inserted from the CSV file, which had 10 entries. 

To find the highest score, the best SQL approach is to use the MAX() function. The correct query would be SELECT MAX(score) FROM scores;. That should return the maximum value in the score column.

I should make sure there are no typos in the query. Also, since the user might expect just the highest value, maybe I can format the response to present it clearly. 

Wait, should I alias the column for better readability? Like SELECT MAX(score) AS highest_score... That way, the result will have a clear column name instead of just MAX(score). 

Yes, that's a good idea. It makes the output more user-friendly. So the final query would be SELECT MAX(score) AS highest_score FROM scores;.

I ne

# Use the GUI

We have explored the capabilities of the Qwen-Agent framework and Qwen models for data manipulations with databases.

Even better, we can easily create a web UI for an agent!

In [20]:
from qwen_agent.gui import WebUI

agent = Assistant(
    name="Qwen Assistant",
    description="I'm a digital assistant powered by Qwen-Agent, ask me anything!",
    llm=llm_cfg,
    function_list=tools
)

WebUI(agent).run()
# Have fun!

2025-06-04 10:37:00,528 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['filesystem', 'sqlite']
2025-06-04 10:37:00,536 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: filesystem
2025-06-04 10:37:01,644 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: sqlite
2025-06-04 10:37:01,947 - mcp_manager.py - 237 - INFO - Failed in listing MCP resource templates: Method not found


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
